Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [3]:
%load_ext sql
%sql sqlite:///HomeW.db

'Connected: @HomeW.db'

In [54]:
%%sql
PRAGMA foreign_keys=ON;
drop table if exists PlayersArchive;
drop table if exists Players;
drop table if exists Teams;
drop table if exists Coaches;
drop table if exists Positions;


create table Coaches (
    id integer not null primary key,
    name varchar(30)
);
create table Positions(
    id integer not null primary key,
    name varchar(30)
);
create table Teams(
    id integer not null primary key,
    name varchar(30),
    location varchar(30),
    coach_id integer,
    foreign key(coach_id) references Coaches(id)
);
create table Players(
    id integer not null primary key,
    name varchar(30),
    age integer,
    position_id integer,
    team_id integer,
    start_date date,
    foreign key(position_id) references Positions(id)
);

create table PlayersArchive(
    player_id integer,
    team_id integer,
    start_date date,
    end_date date,
    foreign key(player_id) references Players(id),
    foreign key(team_id) references Teams(id)
);


 * sqlite:///HomeW.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [55]:
%%sql
insert into Positions (name) values ("Goalkeeper"),("Full-back"),("Wing-back"),("Forward")

 * sqlite:///HomeW.db
4 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [4]:
import sqlite3
db=sqlite3.connect('HomeW.db')

In [22]:
db.close()

In [13]:
def add_coach(name):
    cursor = db.cursor()
    cursor1 = db.cursor()
    cursor1.execute("select id from Coaches where name == ?",(name,))
    try:
        r = cursor1.fetchone()[0]
        print("Table contains this Coach with id = ",r)
    except TypeError:
        cursor.execute("insert into Coaches(name) values(?)",(name,))
        db.commit()
    

In [14]:
add_coach("Coach_0")
add_coach("Coach_2")
add_coach("Coach_3")

Table contains this Coach with id =  1
Table contains this Coach with id =  2
Table contains this Coach with id =  3


In [36]:
%%sql
select * from Coaches

 * sqlite:///HomeW.db
Done.


id,name
1,Coach_0
2,Coach_2
3,Coach_3


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [15]:
def add_team(name,location,coach_name):
    cursor = db.cursor()
    query = "insert into Teams(name,location,coach_id) values('{0}','{1}',(select id from Coaches where name = '{2}'))".format(name,location,coach_name)
    cursor.execute(query)
    db.commit()

In [16]:
add_team("team_1","Moscow","Coach_0")
add_team("team_2","Moscow","Coach_2")
add_team("team_3","Moscow","Coach_3")

# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [17]:
def update_team(team_name,coach_name=None,location=None):
    cursor = db.cursor()
    if coach_name != None:
        query = "update Teams set coach_id = (select id from Coaches where name = '{0}') where name = '{1}'".format(coach_name,team_name)
        cursor.execute(query)
        db.commit()
    if location != None:
        query = "update Teams set location =  '{0}' where name = '{1}' ".format(location,team_name)
        cursor.execute(query)
        db.commit()
    

In [18]:
update_team("team_2",coach_name="Coach_2")
update_team("team_1",location="New York")

In [19]:
%%sql
 select *from Teams

 * sqlite:///HomeW.db
Done.


id,name,location,coach_id
1,team_1,New York,1
2,team_2,Moscow,2
3,team_3,Moscow,3
4,team_1,New York,1
5,team_2,Moscow,2
6,team_3,Moscow,3


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [31]:
def add_player(name,age,position,team_name,date=None):
    cursor= db.cursor()
    if(age > 10 and age <50):
        p_id = None
        t_id = None
        cursor_p = db.cursor()
        query_p = "select id from Positions where name ='{}' ".format(position)
        cursor_p.execute(query_p)
        try:
            p_id= cursor_p.fetchone()[0]
        except:
            print("Wrong position")
            return;
        cursor_t = db.cursor()
        query_t = "select id from Teams where name = '{}'".format(team_name)
        cursor_t.execute(query_t)
        try:
            t_id= cursor_t.fetchone()[0]
        except:
            print("Wrong team")
            return;
        if date == None:
            cursor_d = db.cursor()
            cursor_d.execute("select date('now')")
            da = cursor_d.fetchone()[0]
            query= "insert into Players(name,age,position_id,team_id,start_date) values('{0}','{1}','{2}','{3}','{4}')".format(name,age,p_id,t_id,da)
        else:
            query= "insert into Players(name,age,position_id,team_id,start_date) values('{0}','{1}','{2}','{3}','{4}')".format(name,age,p_id,t_id,date)
        cursor.execute(query)
        db.commit()
    else:
        print("Wrong age")

In [33]:
add_player("Player_1",11,"Goalkeeper","team_1")
add_player("Player_3",45,"Goalkeeper","team_2","2017-11-23")

In [34]:
%%sql
select * from Players

 * sqlite:///HomeW.db
Done.


id,name,age,position_id,team_id,start_date
1,Player_1,10,1,1,None
2,Player_1,11,Goalkeeper,team_1,None
3,Player_1,11,Goalkeeper,team_1,None
4,Player_1,11,None,None,None
5,Player_1,11,1,1,None
6,Player_1,11,1,1,(select date(now))
7,Player_1,11,1,1,2018-12-14
8,Player_1,11,1,1,2018-12-14
9,Player_1,11,1,1,2018-12-14
10,Player_1,11,1,1,2018-12-14


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [44]:
def change_team(name,team_name,end_date):
    cursor1 = db.cursor()
    query1 = "select team_id,start_date,id from Players where name = '{}'".format(name)
    cursor1.execute(query1)
    team_id = ""
    start_date = ""
    player_id = ""
    try:
        data = cursor1.fetchone()
        team_id = data[0]
        start_date = data[1]
        player_id = data[2]
    except:
        print("Error")
        return
    print(start_date)
    cursor2 = db.cursor()
    query2 = "update Players set team_id = (select id from Teams where name = '{0}') where name = '{1}'".format(team_name,name)
    cursor2.execute(query2)
    try:
        pass
    except:
        pass
    cursor3= db.curosr()
    query3 = "insert into Players_archive(player_id,team_id,start_date,end_date) values ('{0}','{1}','{2}','{3}')".format(player_id,team_id,start_date,end_date)
    cursor3.execute(query3)
    db.commit()

In [45]:
change_team("Player_3","asd","asd")

2017-11-23
